In [1]:
import os
import pathlib
import shutil
import json
import yaml
import locale
import argparse

import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

locale.getpreferredencoding = lambda: "UTF-8"

Make subset data

In [2]:
import os
import random
import shutil

def create_subset(images_dir, labels_dir, output_images_dir, output_labels_dir, sample_size):
    if not os.path.exists(output_images_dir):
        os.makedirs(output_images_dir)
    if not os.path.exists(output_labels_dir):
        os.makedirs(output_labels_dir)
    
    files = os.listdir(images_dir)
    random.shuffle(files)
    
    sample_files = files[:sample_size]
    
    for file_name in sample_files:
        image_path = os.path.join(images_dir, file_name)
        label_path = os.path.join(labels_dir, os.path.splitext(file_name)[0] + '.txt')  # Assuming label files have the same name with .txt extension
        
        output_image_path = os.path.join(output_images_dir, file_name)
        output_label_path = os.path.join(output_labels_dir, os.path.splitext(file_name)[0] + '.txt')
        
        shutil.copy(image_path, output_image_path)
        shutil.copy(label_path, output_label_path)

def create_subset_for_all(base_input_dir, base_output_dir, train_size, val_size, test_size):
    sets = [
        ('train', train_size),
        ('val', val_size),
        ('test', test_size)
    ]
    
    for set_name, size in sets:
        images_dir = os.path.join(base_input_dir, set_name, 'images')
        labels_dir = os.path.join(base_input_dir, set_name, 'labels')
        
        output_images_dir = os.path.join(base_output_dir, set_name, 'images')
        output_labels_dir = os.path.join(base_output_dir, set_name, 'labels')
        
        create_subset(images_dir, labels_dir, output_images_dir, output_labels_dir, size)

# Define the base directories for the input and output datasets
base_input_dir = r"C:/Mansura/UTI/data/DATA-UTI-LR/Data"
base_output_dir = r"C:/Mansura/UTI/data/DATA-UTI-LR/Sample-Data"

# Define the sample sizes for each set
train_sample_size = 500
val_sample_size = 100
test_sample_size = 200

# Create subsets of the dataset
create_subset_for_all(base_input_dir, base_output_dir, train_sample_size, val_sample_size, test_sample_size)


CLAHE and GAMMA Correction

In [3]:
import os
import cv2
import numpy as np

def clahe_function(img):
    # Create CLAHE object
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))

    # Convert BGR image to LAB color space
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    # Split LAB image into channels
    lab_planes = list(cv2.split(lab_img))

    # Apply CLAHE to the L channel
    clahe_img = clahe.apply(lab_planes[0])

    # Merge the CLAHE-enhanced L channel with the original A and B channels
    lab_planes[0] = clahe_img
    updated_lab_img = cv2.merge(lab_planes)

    # Convert LAB image back to BGR color space
    clahe_img_bgr = cv2.cvtColor(updated_lab_img, cv2.COLOR_LAB2BGR)

    return clahe_img_bgr

def gamma_correction(img, gamma=1.5):
    # Apply gamma correction
    gamma_corrected = np.power(img / 255.0, gamma) * 255.0
    gamma_corrected = np.uint8(gamma_corrected)
    return gamma_corrected

def process_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for file_name in os.listdir(input_dir):
        input_image_path = os.path.join(input_dir, file_name)
        output_image_path = os.path.join(output_dir, file_name)

        if not os.path.isfile(input_image_path):
            print(f"Skipping non-file: {input_image_path}")
            continue

        try:
            # Read the image
            img = cv2.imread(input_image_path)

            if img is None:
                print(f"Failed to load image: {input_image_path}")
                continue

            # Apply CLAHE
            img = clahe_function(img)

            # Apply gamma correction
            img = gamma_correction(img)

            # Save the processed image
            cv2.imwrite(output_image_path, img)

        except Exception as e:
            print(f"Exception processing image {input_image_path}: {e}")

# Base directory where your input images are stored
base_input_dir = r"C:/Mansura/UTI/data/External-Validation/UMID"
# Base directory where you want to save the processed images
base_output_dir = r"C:/Mansura/UTI/data/External-Validation/UTI-UMID"

# Define the sets and their respective paths
#sets = ['train', 'val', 'test']
sets = ['train']

for set_name in sets:
    input_dir = os.path.join(base_input_dir, set_name, 'images')
    output_dir = os.path.join(base_output_dir, set_name, 'images')
    process_images(input_dir, output_dir)


In [5]:
import os
import cv2
import numpy as np

def clahe_function(img):
    # Create CLAHE object
    clahe = cv2.createCLAHE(clipLimit=3, tileGridSize=(8, 8))

    # Convert BGR image to LAB color space
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    # Split LAB image into channels
    lab_planes = list(cv2.split(lab_img))

    # Apply CLAHE to the L channel
    clahe_img = clahe.apply(lab_planes[0])

    # Merge the CLAHE-enhanced L channel with the original A and B channels
    lab_planes[0] = clahe_img
    updated_lab_img = cv2.merge(lab_planes)

    # Convert LAB image back to BGR color space
    clahe_img_bgr = cv2.cvtColor(updated_lab_img, cv2.COLOR_LAB2BGR)

    return clahe_img_bgr

def gamma_correction(img, gamma=1.2):
    # Apply gamma correction
    gamma_corrected = np.power(img / 255.0, gamma) * 255.0
    gamma_corrected = np.uint8(gamma_corrected)
    return gamma_corrected

def process_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for file_name in os.listdir(input_dir):
        input_image_path = os.path.join(input_dir, file_name)
        output_image_path = os.path.join(output_dir, file_name)

        if not os.path.isfile(input_image_path):
            print(f"Skipping non-file: {input_image_path}")
            continue

        try:
            # Read the image
            img = cv2.imread(input_image_path)

            if img is None:
                print(f"Failed to load image: {input_image_path}")
                continue

            # Apply CLAHE
            img = clahe_function(img)

            # Apply gamma correction
            img = gamma_correction(img)

            # Save the processed image
            cv2.imwrite(output_image_path, img)

        except Exception as e:
            print(f"Exception processing image {input_image_path}: {e}")

# Base directory where your input images are stored
base_input_dir = r"C:/Mansura/UTI/data/DATA-UTI-LR/Sample-Data"
# Base directory where you want to save the processed images
base_output_dir = r"C:/Mansura/UTI/data/DATA-UTI-LR/CG-Sample-Data"

# Define the sets and their respective paths
sets = ['train', 'val', 'test']

for set_name in sets:
    input_dir = os.path.join(base_input_dir, set_name, 'images')
    output_dir = os.path.join(base_output_dir, set_name, 'images')
    process_images(input_dir, output_dir)


In [1]:
import os
import cv2
import numpy as np

def clahe_function(img, clip_limit=2.5, tile_grid_size=(8, 8)):
    # Create CLAHE object
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

    # Convert BGR image to LAB color space
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    # Split LAB image into channels
    lab_planes = list(cv2.split(lab_img))

    # Apply CLAHE to the L channel
    clahe_img = clahe.apply(lab_planes[0])

    # Merge the CLAHE-enhanced L channel with the original A and B channels
    lab_planes[0] = clahe_img
    updated_lab_img = cv2.merge(lab_planes)

    # Convert LAB image back to BGR color space
    clahe_img_bgr = cv2.cvtColor(updated_lab_img, cv2.COLOR_LAB2BGR)

    return clahe_img_bgr

def gamma_correction(img, gamma=1.2):
    # Apply gamma correction
    gamma_corrected = np.power(img / 255.0, gamma) * 255.0
    gamma_corrected = np.uint8(gamma_corrected)
    return gamma_corrected

def denoise_image(img):
    # Apply denoising
    denoised_img = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)
    return denoised_img

def sharpen_image(img):
    # Sharpening kernel
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    sharpened_img = cv2.filter2D(img, -1, kernel)
    return sharpened_img

def process_images(input_dir, output_dir, clip_limit=2.5, tile_grid_size=(8, 8), gamma=1.2):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for file_name in os.listdir(input_dir):
        input_image_path = os.path.join(input_dir, file_name)
        output_image_path = os.path.join(output_dir, file_name)

        if not os.path.isfile(input_image_path):
            print(f"Skipping non-file: {input_image_path}")
            continue

        try:
            # Read the image
            img = cv2.imread(input_image_path)

            if img is None:
                print(f"Failed to load image: {input_image_path}")
                continue

            # Apply CLAHE
            img = clahe_function(img, clip_limit=clip_limit, tile_grid_size=tile_grid_size)

            # Apply gamma correction
            img = gamma_correction(img, gamma=gamma)

            # Optionally apply denoising
            #img = denoise_image(img)

            # Optionally apply sharpening
            #img = sharpen_image(img)

            # Save the processed image
            cv2.imwrite(output_image_path, img)

        except Exception as e:
            print(f"Exception processing image {input_image_path}: {e}")

# Base directory where your input images are stored
base_input_dir = r"C:/Mansura/UTI/data/DATA-UTI-LR/Data"
# Base directory where you want to save the processed images
base_output_dir = r"C:/Mansura/UTI/data/DATA-UTI-LR/CG-Data"

# Define the sets and their respective paths
sets = ['train', 'val', 'test']

# Parameters for CLAHE and Gamma correction
clip_limit = 2.5  # Adjust this value as needed
tile_grid_size = (8, 8)  # Adjust this value as needed
gamma = 1.2  # Adjust this value as needed

for set_name in sets:
    input_dir = os.path.join(base_input_dir, set_name, 'images')
    output_dir = os.path.join(base_output_dir, set_name, 'images')
    process_images(input_dir, output_dir, clip_limit=clip_limit, tile_grid_size=tile_grid_size, gamma=gamma)
    

In [7]:
import os
import cv2
import numpy as np

def clahe_function(img):
    # Create CLAHE object
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))

    # Convert BGR image to LAB color space
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    # Split LAB image into channels
    lab_planes = list(cv2.split(lab_img))

    # Apply CLAHE to the L channel
    clahe_img = clahe.apply(lab_planes[0])

    # Merge the CLAHE-enhanced L channel with the original A and B channels
    lab_planes[0] = clahe_img
    updated_lab_img = cv2.merge(lab_planes)

    # Convert LAB image back to BGR color space
    clahe_img_bgr = cv2.cvtColor(updated_lab_img, cv2.COLOR_LAB2BGR)

    return clahe_img_bgr

def gamma_correction(img, gamma_bg=1.5, gamma_obj=0.7):
    # Create a grayscale image to identify the background and objects
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Threshold the image to create a mask
    _, mask = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
    
    # Normalize the mask to [0, 1]
    mask = mask / 255.0
    
    # Apply gamma correction for the background
    img_bg = np.power(img / 255.0, gamma_bg) * 255.0
    
    # Apply gamma correction for the objects
    img_obj = np.power(img / 255.0, gamma_obj) * 255.0
    
    # Combine the background and object images using the mask
    gamma_corrected = img_bg * (1 - mask[..., np.newaxis]) + img_obj * mask[..., np.newaxis]
    gamma_corrected = np.uint8(gamma_corrected)
    
    return gamma_corrected

def process_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for file_name in os.listdir(input_dir):
        input_image_path = os.path.join(input_dir, file_name)
        output_image_path = os.path.join(output_dir, file_name)

        if not os.path.isfile(input_image_path):
            print(f"Skipping non-file: {input_image_path}")
            continue

        try:
            # Read the image
            img = cv2.imread(input_image_path)

            if img is None:
                print(f"Failed to load image: {input_image_path}")
                continue

            # Apply CLAHE
            img = clahe_function(img)

            # Apply gamma correction
            img = gamma_correction(img)

            # Save the processed image
            cv2.imwrite(output_image_path, img)

        except Exception as e:
            print(f"Exception processing image {input_image_path}: {e}")

# Base directory where your input images are stored
base_input_dir = r"C:/Mansura/UTI/data/External-Validation/UMID"
# Base directory where you want to save the processed images
base_output_dir = r"C:/Mansura/UTI/data/External-Validation/UTI-UMID9"

# Define the sets and their respective paths
sets = ['train']

for set_name in sets:
    input_dir = os.path.join(base_input_dir, set_name, 'images')
    output_dir = os.path.join(base_output_dir, set_name, 'images')
    process_images(input_dir, output_dir)


Histogram Equalization RGB

In [3]:
import os
import cv2

def apply_histogram_equalization(input_folder, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List all files in the input folder
    image_files = os.listdir(input_folder)
    
    for filename in image_files:
        # Read the image
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        
        # Check if the file is an image
        if image is not None:
            # Split the image into RGB channels
            b, g, r = cv2.split(image)
            
            # Apply histogram equalization to each channel
            b_eq = cv2.equalizeHist(b)
            g_eq = cv2.equalizeHist(g)
            r_eq = cv2.equalizeHist(r)
            
            # Merge the equalized channels
            equalized_image = cv2.merge((b_eq, g_eq, r_eq))
            
            # Save the processed image
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, equalized_image)

            #print(f"Processed: {filename}")

# Specify input and output folders
input_folder = "C:/Mansura/UTI/data/DATA-UTI-LR/Sample-Data/val/images"
output_folder = "C:/Mansura/UTI/data/DATA-UTI-LR/HE-Sample-Data/val/images"

# Apply histogram equalization to images in the input folder and save them to the output folder
apply_histogram_equalization(input_folder, output_folder)
